In [1]:
import torch
import yt_dlp as youtube_dl
from transformers import pipeline
from transformers.pipelines.audio_utils import ffmpeg_read
import tempfile
import os
from sentence_transformers import SentenceTransformer

In [3]:
MODEL_NAME = "openai/whisper-small"
BATCH_SIZE = 8
FILE_LIMIT_MB = 1000
YT_LENGTH_LIMIT_S = 3600  # limit to 1 hour YouTube files

device = "cpu"

In [4]:
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
yt_links = dict = {
    'https://www.youtube.com/watch?v=GKof6sDt3FM': 9, 
    'https://www.youtube.com/watch?v=fc_oVKEAQ9g': 8,
    'https://www.youtube.com/watch?v=GW1Oqntimn0': 5,
    'https://www.youtube.com/watch?v=lzt0IsSkBsg': 2,
    'https://www.youtube.com/watch?v=P-g4f5FrDSE': 7,
    'https://www.youtube.com/watch?v=P-g4f5FrDSE': 1,
    'https://www.youtube.com/watch?v=uq83YeThvxU': 1, 
    }

In [6]:
def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

In [7]:
def download_yt_audio(yt_url, filename):
    info_loader = youtube_dl.YoutubeDL()
    
    info = info_loader.extract_info(yt_url, download=False)

    
    file_length = info["duration_string"]
    file_h_m_s = file_length.split(":")
    file_h_m_s = [int(sub_length) for sub_length in file_h_m_s]
    
    if len(file_h_m_s) == 1:
        file_h_m_s.insert(0, 0)
    if len(file_h_m_s) == 2:
        file_h_m_s.insert(0, 0)
    file_length_s = file_h_m_s[0] * 3600 + file_h_m_s[1] * 60 + file_h_m_s[2]
    
    if file_length_s > YT_LENGTH_LIMIT_S:
        yt_length_limit_hms = time.strftime("%HH:%MM:%SS", time.gmtime(YT_LENGTH_LIMIT_S))
        file_length_hms = time.strftime("%HH:%MM:%SS", time.gmtime(file_length_s))
        raise gr.Error(f"Maximum YouTube length is {yt_length_limit_hms}, got {file_length_hms} YouTube video.")
    
    ydl_opts = {"outtmpl": filename, "format": "worstvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"}
    
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([yt_url])


In [8]:
def yt_transcribe(yt_url, task, max_filesize=75.0):
    html_embed_str = _return_yt_html_embed(yt_url)

    with tempfile.TemporaryDirectory() as tmpdirname:
        filepath = os.path.join(tmpdirname, "video.mp4")
        download_yt_audio(yt_url, filepath)
        with open(filepath, "rb") as f:
            inputs = f.read()

    inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
    inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}

    text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]

    return html_embed_str, text

In [9]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

'https://www.youtube.com/watch?v=GKof6sDt3FM'

In [26]:
#strings = []
#embeddings = []
remaining_links = dict(list(yt_links.items())[2:])
for link in remaining_links:
    
    str = yt_transcribe(link, 'transcribe')
    strings.append(str)

    emb = embedding_model.encode(str)
    embeddings.append(emb)

[youtube] Extracting URL: https://www.youtube.com/watch?v=GW1Oqntimn0
[youtube] GW1Oqntimn0: Downloading webpage
[youtube] GW1Oqntimn0: Downloading ios player API JSON
[youtube] GW1Oqntimn0: Downloading android player API JSON


[youtube] GW1Oqntimn0: Downloading player 45986ce4
[youtube] GW1Oqntimn0: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=GW1Oqntimn0
[youtube] GW1Oqntimn0: Downloading webpage
[youtube] GW1Oqntimn0: Downloading ios player API JSON
[youtube] GW1Oqntimn0: Downloading android player API JSON


[youtube] GW1Oqntimn0: Downloading m3u8 information
[info] GW1Oqntimn0: Downloading 1 format(s): 602+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 112
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp08fzrdao\video.f602.mp4
[download] 100% of    1.75MiB in 00:00:07 at 234.83KiB/s                  
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp08fzrdao\video.f140.m4a
[download] 100% of    9.58MiB in 00:00:06 at 1.46MiB/s     
[Merger] Merging formats into "C:\Users\alicj\AppData\Local\Temp\tmp08fzrdao\video.mp4"
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp08fzrdao\video.f140.m4a (pass -k to keep)
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp08fzrdao\video.f602.mp4 (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=lzt0IsSkBsg
[youtube] lzt0IsSkBsg: Downloading webpage
[youtube] lzt0IsSkBsg: Downloading ios player API JSON
[youtube] lzt0IsSkBsg: Downloading android player API JSON


[youtube] lzt0IsSkBsg: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=lzt0IsSkBsg
[youtube] lzt0IsSkBsg: Downloading webpage
[youtube] lzt0IsSkBsg: Downloading ios player API JSON
[youtube] lzt0IsSkBsg: Downloading android player API JSON


[youtube] lzt0IsSkBsg: Downloading m3u8 information
[info] lzt0IsSkBsg: Downloading 1 format(s): 602+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 142
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmpjdghdkj7\video.f602.mp4
[download] 100% of    1.82MiB in 00:00:10 at 170.61KiB/s                 
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmpjdghdkj7\video.f140.m4a
[download] 100% of   11.97MiB in 00:00:08 at 1.47MiB/s     
[Merger] Merging formats into "C:\Users\alicj\AppData\Local\Temp\tmpjdghdkj7\video.mp4"
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmpjdghdkj7\video.f602.mp4 (pass -k to keep)
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmpjdghdkj7\video.f140.m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=P-g4f5FrDSE
[youtube] P-g4f5FrDSE: Downloading webpage
[youtube] P-g4f5FrDSE: Downloading ios player API JSON
[youtube] P-g4f5FrDSE: Downloading android player API JSON


[youtube] P-g4f5FrDSE: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=P-g4f5FrDSE
[youtube] P-g4f5FrDSE: Downloading webpage
[youtube] P-g4f5FrDSE: Downloading ios player API JSON
[youtube] P-g4f5FrDSE: Downloading android player API JSON


[youtube] P-g4f5FrDSE: Downloading player d0ea0c5b
[youtube] P-g4f5FrDSE: Downloading m3u8 information
[info] P-g4f5FrDSE: Downloading 1 format(s): 602+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 203
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp3069jonp\video.f602.mp4
[download] 100% of    3.08MiB in 00:00:27 at 115.38KiB/s                 
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp3069jonp\video.f140.m4a
[download] 100% of   15.54MiB in 00:00:04 at 3.21MiB/s     
[Merger] Merging formats into "C:\Users\alicj\AppData\Local\Temp\tmp3069jonp\video.mp4"
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp3069jonp\video.f140.m4a (pass -k to keep)
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp3069jonp\video.f602.mp4 (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=uq83YeThvxU
[youtube] uq83YeThvxU: Downloading webpage
[youtube] uq83YeThvxU: Downloading ios player API JSON
[youtube]

[youtube] uq83YeThvxU: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=uq83YeThvxU
[youtube] uq83YeThvxU: Downloading webpage
[youtube] uq83YeThvxU: Downloading ios player API JSON
[youtube] uq83YeThvxU: Downloading android player API JSON


[youtube] uq83YeThvxU: Downloading m3u8 information
[info] uq83YeThvxU: Downloading 1 format(s): 602+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 114
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmpf6jowlck\video.f602.mp4
[download] 100% of    1.74MiB in 00:00:46 at 38.69KiB/s                  
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmpf6jowlck\video.f140.m4a
[download] 100% of    9.61MiB in 00:00:02 at 3.99MiB/s   
[Merger] Merging formats into "C:\Users\alicj\AppData\Local\Temp\tmpf6jowlck\video.mp4"
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmpf6jowlck\video.f140.m4a (pass -k to keep)
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmpf6jowlck\video.f602.mp4 (pass -k to keep)


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Replace this with the URL you want to scrape
url = 'http://example.com'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# This pattern matches both 'www.youtube.com' and 'youtu.be' URLs
youtube_url_pattern = re.compile(r'(https?://(?:www\.)?youtube\.com/watch\?v=[\w-]+)|(https?://youtu\.be/[\w-]+)')

# Find all <a> tags, then filter by href attribute matching the YouTube URL pattern
youtube_links = [a['href'] for a in soup.find_all('a', href=True) if youtube_url_pattern.search(a['href'])]

for link in youtube_links:
    print(link)


In [17]:
link = list(yt_links.items())[0][0]

In [27]:
yt_links

{'https://www.youtube.com/watch?v=GKof6sDt3FM': 9,
 'https://www.youtube.com/watch?v=fc_oVKEAQ9g': 8,
 'https://www.youtube.com/watch?v=GW1Oqntimn0': 5,
 'https://www.youtube.com/watch?v=lzt0IsSkBsg': 2,
 'https://www.youtube.com/watch?v=P-g4f5FrDSE': 1,
 'https://www.youtube.com/watch?v=uq83YeThvxU': 1}

In [32]:
link = 'https://www.youtube.com/watch?v=_ID4uPKn40w&t=4s'

In [33]:
str = yt_transcribe(link, 'transcribe')

[youtube] Extracting URL: https://www.youtube.com/watch?v=_ID4uPKn40w&t=4s
[youtube] _ID4uPKn40w: Downloading webpage
[youtube] _ID4uPKn40w: Downloading ios player API JSON
[youtube] _ID4uPKn40w: Downloading android player API JSON


[youtube] _ID4uPKn40w: Downloading player 7ebf4817
[youtube] _ID4uPKn40w: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=_ID4uPKn40w&t=4s
[youtube] _ID4uPKn40w: Downloading webpage
[youtube] _ID4uPKn40w: Downloading ios player API JSON
[youtube] _ID4uPKn40w: Downloading android player API JSON


[youtube] _ID4uPKn40w: Downloading m3u8 information
[info] _ID4uPKn40w: Downloading 1 format(s): 602+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 36
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp1vwmnbvp\video.f602.mp4
[download] 100% of  543.79KiB in 00:00:32 at 16.52KiB/s                  
[download] Destination: C:\Users\alicj\AppData\Local\Temp\tmp1vwmnbvp\video.f140.m4a
[download] 100% of    2.97MiB in 00:00:01 at 2.09MiB/s   
[Merger] Merging formats into "C:\Users\alicj\AppData\Local\Temp\tmp1vwmnbvp\video.mp4"
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp1vwmnbvp\video.f602.mp4 (pass -k to keep)
Deleting original file C:\Users\alicj\AppData\Local\Temp\tmp1vwmnbvp\video.f140.m4a (pass -k to keep)


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [35]:
str

('<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/_ID4uPKn40w&t=4s"> </iframe> </center>',
 " Peppa Pig Tails That car went really fast. Don't worry, this is why it's very important to stop. Look, listen and always wait for the green man before crossing the road. The Green Man Remember what we learned, children? Stop, look and listen All clear, no cause Let's go. It's safe to cross now. Peppa and her playgroup are learning about road safety today. Hello, Daddy. Hello, Peppa. Peppa's playgroup! Hello Daddy Pig! Oh, green light! Well, see you later Peppa! Thank you for helping us Police Officer Panda! All in a day's work! Peppa is helping Police Officer Panda make sure everyone follows road safety rules. No, no, no. Emily, you need to stay on your side of the road. Sorry Peppa, my wheel is stuck. We'll come help you! Wee-oo, wee-oo, wee-oo! We're keeping the road safe for cars and people. Pedro hasn't noticed that Green Man has turned to red. Pedro hasn't not

In [34]:
emb = embedding_model.encode(str)

In [45]:
import pandas as pd
import numpy as np

In [48]:
embedding = np.append(emb[1], [8])

In [50]:
pd.DataFrame([embedding])

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,384
0,0.00851,-0.025821,0.015261,-0.040453,0.05157,0.031697,0.086667,0.02466,0.006148,0.011284,...,-0.027371,-0.032646,-0.000516,-0.007383,0.037214,0.009844,0.065942,0.048913,0.001026,8.0


In [41]:
emb[1]

array([ 8.50950368e-03, -2.58213710e-02,  1.52609302e-02, -4.04529013e-02,
        5.15700169e-02,  3.16970386e-02,  8.66668373e-02,  2.46598739e-02,
        6.14783447e-03,  1.12838307e-02,  6.72722161e-02, -2.69743260e-02,
        1.96951535e-02,  2.77098045e-02, -5.46237864e-02,  4.63798754e-02,
        6.65528188e-03,  2.59582270e-02, -6.95972368e-02, -1.25428280e-02,
       -1.75009388e-02, -1.78021789e-02,  3.64594907e-02,  6.69801831e-02,
       -1.82968512e-01,  5.96265011e-02, -1.65427756e-03,  5.12275845e-02,
       -5.81769422e-02, -7.59152770e-02, -4.77800779e-02,  3.81396292e-03,
        3.75223570e-02, -2.20142882e-02, -4.73976843e-02, -2.55290959e-02,
        1.23119690e-02, -8.48205294e-03,  1.14649057e-01,  3.35947834e-02,
       -3.79976556e-02, -2.12080497e-02, -1.12434411e-02, -2.46901587e-02,
        5.45684770e-02, -1.81832574e-02,  3.40077654e-02, -7.20066344e-03,
        9.88185778e-02, -1.52009100e-01,  4.20196261e-03,  5.92234544e-02,
       -7.92676117e-03,  